![Texto alternativo](../tarea-Simulacion/imagen1.png)


# Paso 1, Importamos Librerias

In [1]:
import numpy as np
from scipy.sparse import diags, kron, identity
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt


# Paso 2, Ponemos los parametros

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros
Lx = 1  # Longitud en x
Ly = 1  # Longitud en y
Tmin = 0  # Tiempo mínimo
Tmax = 10  # Tiempo máximo

Nx = 3  # Número de puntos en x
Ny = 3  # Número de puntos en y
Nt = 11  # Número de pasos de tiempo

dx = Lx / (Nx - 1)  # Paso en x
dy = Ly / (Ny - 1)  # Paso en y
dt = (Tmax-Tmin) / (Nt - 1)  # Paso de tiempo

alpha = 0.01  # Coeficiente de difusión térmica

# Crear la malla 2D
x = np.linspace(0, Lx, Nx) # Crea un vector [0, 0.5, 1]
y = np.linspace(0, Ly, Ny)
t = np.linspace(0, Tmax, Nt)

# en lugar de T = np.zeros((Nx, Ny))
T = np.zeros((Nx, Ny, Nt))   # T[i,j,n]

# la condición inicial pasa a T[:,:,0]
for i in range(1, Nx-1):
    for j in range(0, Ny):
        T[i, j, 0] = 100*np.sin(np.pi*x[i])*np.sin(np.pi*y[j])




t= [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


![Texto alternativo](../tarea-Simulacion/imagen2.png)


In [ ]:

# Matrices de coeficientes para el sistema implícito en cada dirección
Ax = np.zeros((Nx-2, Nx-2))  # Matriz para la dirección x
Ay = np.zeros((Ny-2, Ny-2))  # Matriz para la dirección y

# Construcción de la matriz Ax para la dirección x
for i in range(Nx-2):  # Ajuste en el rango
    if i > 0:
        Ax[i, i-1] = -rx
    Ax[i, i] = 1 + 2*rx
    if i < Nx-3:  # Ajuste en el rango
        Ax[i, i+1] = -rx

# Condiciones en los bordes de la matriz Ax
Ax[0, 0] = 1
Ax[Nx-3, Nx-3] = 1

# Construcción de la matriz Ay para la dirección y
for j in range(Ny-2):  # Ajuste en el rango
    if j > 0:
        Ay[j, j-1] = -ry
    Ay[j, j] = 1 + 2*ry
    if j < Ny-3:  # Ajuste en el rango
        Ay[j, j+1] = -ry

# Condiciones en los bordes de la matriz Ay
Ay[0, 0] = 1
Ay[Ny-3, Ny-3] = 1

# Condiciones de frontera (Neumann en los bordes)
# En x=0, x=Lx (Neumann)
T[0, :, :] = T[1, :, :]
T[Nx-1, :, :] = T[Nx-2, :, :]

# En y=0, y=Ly (Neumann)
T[:, 0, :] = T[:, 1, :]
T[:, Ny-1, :] = T[:, Ny-2, :]

# Resolución implícita en cada paso de tiempo
for n in range(0, Nt-1):
    # Resolver en cada fila (dirección x)
    for j in range(Ny):
        T[1:Nx-1, j, n+1] = np.linalg.solve(Ax, T[1:Nx-1, j, n])

    # Resolver en cada columna (dirección y)
    for i in range(Nx):
        T[i, 1:Ny-1, n+1] = np.linalg.solve(Ay, T[i, 1:Ny-1, n])

# Graficar los resultados
X, Y = np.meshgrid(x, y)
plt.figure(figsize=(8,6))
plt.contourf(X, Y, T[:,:,Nt-1], 20, cmap='inferno')
plt.colorbar()
plt.title(f'Temperatura en t={Tmax} s')
plt.xlabel('x')
plt.ylabel('y')
plt.show()
